Esta nota utiliza métodos vistos en [1.5.Integracion_numerica](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.5.Integracion_numerica.ipynb)

**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_r_kernel_local -p 8888:8888 -d palmoreck/jupyterlab_r_kernel:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_r_kernel_local
```


Documentación de la imagen de docker `palmoreck/jupyterlab_r_kernel:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/r_kernel).

Instalamos Rcpp:

In [3]:
install.packages("Rcpp",lib="/usr/local/lib/R/site-library/",
                 repos="https://cran.itam.mx/",verbose=TRUE)

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

foundpkgs: Rcpp, /tmp/RtmptkKQTl/downloaded_packages/Rcpp_1.0.3.tar.gz

files: /tmp/RtmptkKQTl/downloaded_packages/Rcpp_1.0.3.tar.gz

1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/RtmptkKQTl/downloaded_packages/Rcpp_1.0.3.tar.gz'



In [6]:
install.packages("microbenchmark",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/",verbose=TRUE)

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

foundpkgs: microbenchmark, /tmp/RtmptkKQTl/downloaded_packages/microbenchmark_1.4-7.tar.gz

files: /tmp/RtmptkKQTl/downloaded_packages/microbenchmark_1.4-7.tar.gz

1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/RtmptkKQTl/downloaded_packages/microbenchmark_1.4-7.tar.gz'



In [1]:
library(Rcpp)
library(microbenchmark)

In [2]:
Rcf1<-function(f,a,b,n){
    #Compute numerical approximation using rectangle or mid-point method in 
    #an interval.
    #Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    #Args:
    #    f (function): function of integrand
    #    a (int): left point of interval
    #    b (int): right point of interval
    #    n (int): number of subintervals
    #Returns:
    #    Rcf (float)
    h_hat<-(b-a)/n
    sum_res<-0
    x<-vapply(0:(n-1),function(j)a+(j+1/2.0)*h_hat,numeric(1))
    for(j in 1:n){
        sum_res<-sum_res+f(x[j])
    }
    h_hat*sum_res
}

In [3]:
f<-function(x)exp(-x^2)

In [4]:
n<-10**6
aprox<-Rcf1(f,0,1,n)

In [5]:
aprox

[1] 0.7468241

**Recuérdese** revisar el error relativo:

In [6]:
err_relativo<-function(aprox,obj)abs(aprox-obj)/abs(obj)

In [7]:
obj<-integrate(Vectorize(f),0,1)

In [8]:
err_relativo(aprox,obj$value)

[1] 6.719397e-14

In [9]:
system.time(Rcf1(f,0,1,n))

   user  system elapsed 
  1.570   0.000   1.572 

In [10]:
Rcf2<-function(f,a,b,n){
    #Compute numerical approximation using rectangle or mid-point method in 
    #an interval.
    #Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    #Args:
    #    f (function): function of integrand
    #    a (int): left point of interval
    #    b (int): right point of interval
    #    n (int): number of subintervals
    #Returns:
    #    Rcf (float)
    h_hat<-(b-a)/n
    x<-vapply(0:(n-1),function(j)a+(j+1/2.0)*h_hat,numeric(1))
    h_hat*sum(f(x))
}

In [11]:
aprox<-Rcf2(f,0,1,n)

In [12]:
aprox

[1] 0.7468241

In [13]:
err_relativo(aprox,obj$value)

[1] 4.102995e-14

In [14]:
system.time(Rcf2(f,0,1,n))

   user  system elapsed 
  0.750   0.000   0.752 

# Hacia la compilación con Rcpp

In [15]:
Rcf3<-function(f,a,b,n){
    #Compute numerical approximation using rectangle or mid-point method in 
    #an interval.
    #Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    #Args:
    #    f (function): function of integrand
    #    a (int): left point of interval
    #    b (int): right point of interval
    #    n (int): number of subintervals
    #Returns:
    #    Rcf (float)
    h_hat<-(b-a)/n
    x<-vapply(0:(n-1),function(j)a+(j+1/2.0)*h_hat,numeric(1))
    h_hat*sum(f(x))
}

In [16]:
aprox<-Rcf3(f,0,1,n)

In [17]:
err_relativo(aprox,obj$value)

[1] 4.102995e-14

In [18]:
system.time(Rcf3(f,0,1,n))

   user  system elapsed 
  0.810   0.010   0.817 

In [19]:
Rcf3b<-function(f,a,b,n){
    #Compute numerical approximation using rectangle or mid-point method in 
    #an interval.
    #Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    #Args:
    #    f (function): function of integrand
    #    a (int): left point of interval
    #    b (int): right point of interval
    #    n (int): number of subintervals
    #Returns:
    #    Rcf (float)
    h_hat<-(b-a)/n
    fx<-f(vapply(0:(n-1),function(j)a+(j+1/2.0)*h_hat,numeric(1)))
    h_hat*sum(fx)
}

In [20]:
aprox<-Rcf3b(f,0,1,n)

In [21]:
err_relativo(aprox,obj$value)

[1] 4.102995e-14

In [22]:
system.time(Rcf3(f,0,1,n))

   user  system elapsed 
  0.760   0.000   0.758 

In [23]:
Rcf4<-function(f,a,b,n){
    #Compute numerical approximation using rectangle or mid-point method in 
    #an interval.
    #Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    #Args:
    #    f (function): function of integrand
    #    a (int): left point of interval
    #    b (int): right point of interval
    #    n (int): number of subintervals
    #Returns:
    #    Rcf (float)
    h_hat<-(b-a)/n
    sum_res<-0
    for(i in 0:(n-1)){
        x<-a+(i+1/2.0)*h_hat
        sum_res<-sum_res+f(x)
    }
    h_hat*sum_res
}

In [24]:
n<-10**6
aprox<-Rcf4(f,0,1,n)

In [25]:
aprox

[1] 0.7468241

In [26]:
err_relativo(aprox,obj$value)

[1] 6.719397e-14

In [27]:
system.time(Rcf4(f,0,1,n))

   user  system elapsed 
  0.560   0.000   0.562 

In [28]:
f_str='double Rcf_Rcpp(double a, double b, int n){
             double h_hat;
             double sum_res=0;
             int i;
             double x;
             h_hat=(b-a)/n;
             for(i=0;i<=n-1;i++){
                    x = a+(i+1/2.0)*h_hat;
                    sum_res+=exp(-pow(x,2));
             }
             return h_hat*sum_res;
        }'

In [29]:
cppFunction(f_str,rebuild=TRUE)

In [30]:
aprox_rcpp<-Rcf_Rcpp(0,1,n)

In [31]:
err_relativo(aprox_rcpp,obj$value)

[1] 6.719397e-14

In [32]:
system.time(Rcf_Rcpp(0,1,n))

   user  system elapsed 
   0.02    0.00    0.02 

In [33]:
mbk<-microbenchmark(
    Rcf1(f,0,1,n),
    Rcf2(f,0,1,n),
    Rcf3(f,0,1,n),
    Rcf3b(f,0,1,n),
    Rcf4(f,0,1,n),
    Rcf_Rcpp(0,1,n), 
    times=10
    )

In [34]:
print(mbk)

Unit: milliseconds
              expr        min         lq       mean    median         uq
  Rcf1(f, 0, 1, n) 1135.19964 1149.82027 1172.46332 1161.1877 1172.04174
  Rcf2(f, 0, 1, n)  669.00516  675.75856  698.36320  692.3119  714.13926
  Rcf3(f, 0, 1, n)  671.57994  680.10762  695.56041  690.4347  709.35516
 Rcf3b(f, 0, 1, n)  670.06599  670.89791  702.90167  682.0686  694.88526
  Rcf4(f, 0, 1, n)  523.72017  539.05730  605.69508  550.2558  580.36165
 Rcf_Rcpp(0, 1, n)   16.57756   16.97325   17.75719   17.0651   18.62889
        max neval
 1286.08346    10
  756.42608    10
  739.47197    10
  856.47839    10
 1086.26442    10
   19.74708    10


In [35]:
a<-0
b<-1
h_hat<-(b-a)/n

Ejemplo de NumericVector

In [36]:
system.time(fx<-f(vapply(0:(n-1),function(j)a+(j+1/2.0)*h_hat,numeric(1))))

   user  system elapsed 
  0.830   0.000   0.833 

In [37]:
f_str2='
double Rcf_Rcpp2(NumericVector f_x,int n,double h_hat){
             double sum_res=0;
             int i;
             for(i=0;i<=n-1;i++){
                    sum_res+=f_x[i];
             }
             return h_hat*sum_res;
        }'

In [38]:
cppFunction(f_str2,rebuild=TRUE)

In [39]:
system.time(Rcf_Rcpp2(fx,n,h_hat))

   user  system elapsed 
  0.000   0.000   0.001 

In [43]:
aprox_rcpp2<-Rcf_Rcpp2(fx,n,h_hat)

In [44]:
err_relativo(aprox_rcpp2,obj$value)

[1] 6.719397e-14

In [47]:
mbk<-microbenchmark(
    Rcf1(f,0,1,n),
    Rcf2(f,0,1,n),
    Rcf3(f,0,1,n),
    Rcf3b(f,0,1,n),
    Rcf4(f,0,1,n),
    Rcf_Rcpp(0,1,n), 
    f(vapply(0:(n-1),function(j)a+(j+1/2.0)*h_hat,numeric(1))),
    times=10
    )

In [48]:
print(mbk)

Unit: milliseconds
                                                                expr        min
                                                    Rcf1(f, 0, 1, n) 1129.45211
                                                    Rcf2(f, 0, 1, n)  668.61445
                                                    Rcf3(f, 0, 1, n)  660.61281
                                                   Rcf3b(f, 0, 1, n)  674.54732
                                                    Rcf4(f, 0, 1, n)  515.94721
                                                   Rcf_Rcpp(0, 1, n)   16.64171
 f(vapply(0:(n - 1), function(j) a + (j + 1/2) * h_hat, numeric(1)))  716.36730
         lq       mean     median         uq        max neval
 1166.04659 1176.09376 1176.38090 1201.51149 1204.37378    10
  678.60933  705.51382  693.36913  737.92405  756.77204    10
  687.66452  699.55605  698.83038  717.39730  725.88417    10
  676.03950  690.30455  687.87424  703.04028  713.51001    10
  535.89703  538.39454  538.352

Llamar funciones en env:

In [51]:
f_str3='RObject fun(double x){
Environment env = Environment::global_env();
Function f=env["f"];
return f(x);
}
'

In [52]:
cppFunction(f_str3,rebuild=TRUE)

In [53]:
fun(1)

[1] 0.3678794

In [54]:
f(1)

[1] 0.3678794

**Referencias**

1. Advanced R
2. http://dirk.eddelbuettel.com/code/rcpp.html
3. https://teuder.github.io/rcpp4everyone_en/
4. http://dirk.eddelbuettel.com/papers/rcpp_rfinance_may2017.pdf